In [2]:
df_final = pd.read_csv("/Users/Christian/Documents/Cand_merc/Data-Science-Project/data_science_project/Beat the bookies/data/Football_data_final_final.csv")

In [3]:
df_final.shape
df_final.columns

Index(['Wk', 'Day', 'Date', 'Time', 'Home', 'Away', 'FTR', 'Attendance',
       'B365H', 'B365D', 'B365A', 'AttackStrengthHome', 'AttackStrengthAway',
       'DefenseWeaknessHome', 'DefenseWeaknessAway', 'Season', 'AvgHomePoints',
       'AvgAwayPoints', 'AvgLosingHomePoints', 'AvgLosingAwayPoints',
       'AvgGoalDiffHome', 'AvgGoalDiffAway', 'HomeWinsRatio', 'HomeDrawsRatio',
       'AwayWinsRatio', 'AwayDrawsRatio', 'AvgHomeCornersLast5',
       'AvgAwayCornersLast5', 'AvgHomeShotsLast5', 'AvgHomeShotsOnTargetLast5',
       'AvgAwayShotsLast5', 'AvgAwayShotsOnTargetLast5', 'elo', 'elo_away',
       'FormHomeTeam', 'FormAwayTeam', 'ProbabilityDraw', 'ProbabilityHomeWin',
       'ProbabilityAwayWin'],
      dtype='object')

In [4]:
df_final.head()

,Wk,Day,Date,Time,Home,Away,FTR,Attendance,B365H,B365D,...,AvgHomeShotsOnTargetLast5,AvgAwayShotsLast5,AvgAwayShotsOnTargetLast5,elo,elo_away,FormHomeTeam,FormAwayTeam,ProbabilityDraw,ProbabilityHomeWin,ProbabilityAwayWin
0,1.0,Fri,2019-08-16,20:00,Athletic Club,Barcelona,H,47693.0,5.25,3.80,...,0.0,0.0,0.0,1722.019409,2001.228638,0.0,0.0,0.226260,0.127955,0.645785
1,1.0,Sat,2019-08-17,16:00,Celta Vigo,Real Madrid,A,23566.0,4.75,4.20,...,0.0,0.0,0.0,1681.163452,1841.848999,0.0,0.0,0.239320,0.266267,0.494412
2,1.0,Sat,2019-08-17,18:00,Valencia,Real Sociedad,D,41846.0,1.66,3.75,...,0.0,0.0,0.0,1813.655029,1719.431152,0.0,0.0,0.229859,0.616701,0.153439
3,1.0,Sat,2019-08-17,19:00,Mallorca,Eibar,H,15127.0,2.80,3.20,...,0.0,0.0,0.0,1566.961060,1712.778198,0.0,0.0,0.240204,0.286297,0.473499
4,1.0,Sat,2019-08-17,20:00,Leganés,Osasuna,A,10020.0,2.00,3.20,...,0.0,0.0,0.0,1670.794800,1641.649048,0.0,0.0,0.237143,0.533531,0.229326


In [5]:
df_final["FTR"].value_counts()

FTR
H    831
A    520
D    509
Name: count, dtype: int64

In [ ]:
def home_away_results(df_final: pd.DataFrame):
    """Returns aggregated home/away results per team"""

    res = (
        pd.melt(df_final.reset_index(),
                id_vars=['Date', 'FTR'],
                value_name='team', var_name='is_home',
                value_vars=['Home', 'Away']))

    res.is_home = res.is_home.replace(['home_team', 'away_team'], ['Home', 'Away'])
    res['win'] = res['lose'] = res['draw'] = 0
    res.loc[(res['is_home'] == 'Home') & (res['FTR'] == 'H'), 'win'] = 1
    res.loc[(res['is_home'] == 'Away') & (res['FTR'] == 'A'), 'win'] = 1
    res.loc[(res['is_home'] == 'Home') & (res['FTR'] == 'A'), 'lose'] = 1
    res.loc[(res['is_home'] == 'Away') & (res['FTR'] == 'H'), 'lose'] = 1
    res.loc[res['FTR'] == 'D', 'draw'] = 1

    g = res.groupby(['team', 'is_home'])
    w = (g.win.agg(['sum', 'mean'])
         .rename(columns={'sum': 'n_win', 'mean': 'win_pct'}))
    l = (g.lose.agg(['sum', 'mean'])
         .rename(columns={'sum': 'n_lose', 'mean': 'lose_pct'}))
    d = (g.draw.agg(['sum', 'mean'])
         .rename(columns={'sum': 'n_draw', 'mean': 'draw_pct'}))

    res = pd.concat([w, l, d], axis=1)
    return res

In [ ]:
results = home_away_results(df_final)
results.head(6)

In [ ]:
g = sns.FacetGrid(results.reset_index(), hue='team', palette='Set2', height=6, aspect=.5)
g.map(sns.pointplot, 'is_home', 'win_pct', order=["Away", "Home"])
g.set_axis_labels('', 'win %');

In [ ]:
g = sns.FacetGrid(results.reset_index(), col='team', col_wrap=5)
g.map(sns.pointplot, 'is_home', 'win_pct', order=["Away", "Home"])
g.set_axis_labels('', 'win %');

In [ ]:
summary_stats = df_final.describe()

In [ ]:
summary_stats = summary_stats.drop(['25%', '50%', '75%'])

In [ ]:
summary_stats.T

In [6]:
targetData = df_final['FTR'].replace({'H':1,'D':0,'A':-1})

C:\Users\Christian\AppData\Local\Temp\ipykernel_7256\3008560993.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  targetData = df_final['FTR'].replace({'H':1,'D':0,'A':-1})


In [ ]:
corr_data = df_final[['B365H', 'B365D', 'B365A',
       'AvgHomePoints', 'AvgAwayPoints', 'AvgGoalDiffHome', 'AvgGoalDiffAway', 'AwayDrawsRatio', 'AvgHomeShotsLast5',
       'AvgHomeShotsOnTargetLast5', 'AvgAwayShotsLast5',
       'AvgAwayShotsOnTargetLast5', 'elo', 'elo_away', 'FormHomeTeam',
       'FormAwayTeam', 'ProbabilityHomeWin', 'ProbabilityAwayWin']]

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

combined_data = pd.concat([corr_data, targetData], axis=1)

# Calculate the correlation matrix
correlation_matrix = combined_data.corr()

# Plot a heatmap of the correlation matrix
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", annot_kws={"size": 10})
plt.title("Correlation Heatmap")
plt.show()

In [ ]:
df_final.to_csv("df_final.csv")

In [7]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [9]:
# Function to calculate VIF
def calculate_vif(df):
    # Add a constant to the model (required for statsmodels)
    X = sm.add_constant(df)
    vif_data = pd.DataFrame()
    vif_data["Feature"] = df.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i + 1) for i in range(len(df.columns))]
    return vif_data

# Calculate VIF for df_final
vif_data = calculate_vif(df_final[['B365H', 'B365D', 'B365A',
       'AvgHomePoints', 'AvgAwayPoints', 'AvgGoalDiffHome', 'AvgGoalDiffAway', 'AwayDrawsRatio', 'AvgHomeShotsLast5',
       'AvgHomeShotsOnTargetLast5', 'AvgAwayShotsLast5',
       'AvgAwayShotsOnTargetLast5', 'elo', 'elo_away', 'FormHomeTeam',
       'FormAwayTeam', 'ProbabilityHomeWin', 'ProbabilityAwayWin']])
print(vif_data)

                      Feature          VIF
0                       B365H     7.329674
1                       B365D     7.294843
2                       B365A    11.185365
3               AvgHomePoints     5.057337
4               AvgAwayPoints     4.413091
5             AvgGoalDiffHome     4.730024
6             AvgGoalDiffAway     3.855302
7              AwayDrawsRatio     1.076416
8           AvgHomeShotsLast5     2.767815
9   AvgHomeShotsOnTargetLast5     3.307350
10          AvgAwayShotsLast5     2.693197
11  AvgAwayShotsOnTargetLast5     3.173969
12                        elo   104.161757
13                   elo_away   103.793592
14               FormHomeTeam     2.750635
15               FormAwayTeam     2.933387
16         ProbabilityHomeWin  1593.450924
17         ProbabilityAwayWin  1000.419332
